# autograd机制

在机器学习中，我们通常使用梯度下降（gradient descent）来更新模型参数从而求解。损失函数关于模型参数的梯度指向一个可以降低损失函数值的方向，我们不断地沿着梯度的方向更新模型从而最小化损失函数。虽然梯度计算比较直观，但对于复杂的模型，例如多达数十层的神经网络，手动计算梯度非常困难。

PyTorch中提供给了我们自动化求导的包 —— autograd

导入之后，**我们用一个 Variable $X$ 对象来包装Tensor变量。然后 $X$.data 就是该Tensor变量，$X$.grad 就是梯度咯**

PyTorch的 Variable 跟 Tensor 有着几乎一样的API，在 Tensor 上进行的操作也基本都能在 Variable 上进行。**区别在于：使用 Variable 对象，你会定义一张动态图，它可以帮助你进行自动求导**

![Variable对象](./images/Variable.png)


In [1]:
# 把必要的包给导入
import torch
from torch.autograd import Variable

如果我们想计算 $f = 3 \times x^3 + 4 \times x^2 + 6$ 的导数，该如何做呢？

In [2]:
def fn(x):
    y = 3 * x.pow(3) + 4 * x.pow(2) + 6
    return y

x1 = Variable(torch.Tensor([1]), requires_grad=True)

y1 = fn(x1)

print(y1)

Variable containing:
 13
[torch.FloatTensor of size 1]



In [3]:
y1.backward() # 自动求导

In [4]:
x1.grad # 查看梯度

Variable containing:
 17
[torch.FloatTensor of size 1]

通过调用 backward() 函数，我们自动求出了在 x = 1 的时候的导数

需要注意的一点是：如果输入的Tensor不是一个标量，矢量（多个值）， 我们在调用backward()之前，需要让**结果变成矢量** 才能求出导数。

如果不将Y的值变成标量，就会报错。（可以尝试把mean()给取消，看看是不是报错了）

In [5]:
x2 = Variable(torch.Tensor([[1, 2], [3, 4]]), requires_grad=True)

y2 = fn(x2).mean() # 将结果变成标量，这样就不会报错了

y2.backward() # 自动求导

In [6]:
x2.grad # 查看梯度

Variable containing:
  4.2500  13.0000
 26.2500  44.0000
[torch.FloatTensor of size 2x2]

In [ ]:
import torch
# M是样本数量，input_size是输入层大小
# hidden_size是隐含层大小，output_size是输出层大小
M, input_size, hidden_size, output_size = 64, 1000, 100, 10

# 生成随机数当作样本
x = torch.randn(M, input_size) #size(64, 1000)
y = torch.randn(M, output_size) #size(64, 10)

# 参数初始化
def init_parameters():
    w1 = torch.randn(input_size, hidden_size)
    w2 = torch.randn(hidden_size, output_size)
    b1 = torch.randn(1, hidden_size)
    b2 = torch.randn(1, output_size)
    return {"w1": w1, "w2":w2, "b1": b1, "b2": b2}

# 定义模型
def model(x, parameters):
    Z1 = x.mm(parameters["w1"]) + parameters["b1"] # 线性层
    A1 = Z1.clamp(min=0) # relu激活函数
    Z2 = A1.mm(parameters["w2"]) + parameters["b2"] #线性层
    # 为了方便反向求导，我们会把当前求得的结果保存在一个cache中
    cache = {"Z1": Z1, "A1": A1}
    return Z2, cache

# 计算损失
def loss_fn(y, y_pred):
    m = y.size()[1] # m个样本
    loss = (y_pred - y).pow(2).sum() # 我们这里直接使用 MSE(均方误差) 作为损失函数
    return loss

# 反向传播，求出梯度
def backpropogation(x, y, y_pred, cache, parameters):
    m = y.size()[0] # m个样本
    # 以下是反向求导的过程：
    d_y_pred = 1/m * (y_pred - y)
    d_w2 = 1/m * cache["A1"].t().mm(d_y_pred)
    d_b2 = 1/m * torch.sum(d_y_pred, 0, keepdim=True)
    d_A1 = d_y_pred.mm(parameters["w2"].t())
    # 对 relu 函数求导: start
    d_Z1 = d_A1.clone()
    d_Z1[cache["Z1"] < 0] = 0
    # 对 relu 函数求导: end
    d_w1 = 1/m * x.t().mm(d_Z1)
    d_b1 = 1/m * torch.sum(d_Z1, 0, keepdim=True)
    grads = {
        "d_w1": d_w1, 
        "d_b1": d_b1, 
        "d_w2": d_w2, 
        "d_b2": d_b2
    }
    return grads

# 更新参数
def update(lr, parameters, grads):
    parameters["w1"] -= lr * grads["d_w1"]
    parameters["w2"] -= lr * grads["d_w2"]
    parameters["b1"] -= lr * grads["d_b1"]
    parameters["b2"] -= lr * grads["d_b2"]
    return parameters

## 设置超参数 ##

learning_rate = 1e-2
EPOCH = 400

# 参数初始化
parameters = init_parameters()

## 开始训练 ##
for t in range(EPOCH):    
    # 向前传播
    y_pred, cache = model(x, parameters)
    
    # 计算损失
    loss = loss_fn(y, y_pred)
    if (t+1) % 50 == 0:
        print(loss)
    # 反向传播
    grads = backpropogation(x, y, y_pred, cache, parameters)
    # 更新参数
    parameters = update(learning_rate, parameters, grads)